In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/MyDrive/Yolo_v6/datasets_usados_estructurados/custom_dataset.zip" 

Se han truncado las últimas 5000 líneas del flujo de salida.
  inflating: content/drive/MyDrive/Yolo_v6/datasets_usados_estructurados/custom_dataset/labels/train/0000322uav0000323_01173_v.txt  
  inflating: content/drive/MyDrive/Yolo_v6/datasets_usados_estructurados/custom_dataset/labels/train/0000423uav0000323_01173_v.txt  
  inflating: content/drive/MyDrive/Yolo_v6/datasets_usados_estructurados/custom_dataset/labels/train/0000205uav0000140_01590_v.txt  
  inflating: content/drive/MyDrive/Yolo_v6/datasets_usados_estructurados/custom_dataset/labels/train/0000354uav0000289_00001_v.txt  
  inflating: content/drive/MyDrive/Yolo_v6/datasets_usados_estructurados/custom_dataset/labels/train/0000024uav0000357_00920_v.txt  
  inflating: content/drive/MyDrive/Yolo_v6/datasets_usados_estructurados/custom_dataset/labels/train/0000300uav0000266_03598_v.txt  
  inflating: content/drive/MyDrive/Yolo_v6/datasets_usados_estructurados/custom_dataset/labels/train/0000081uav0000315_00000_v.txt  
  inflat

In [2]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v6.2-224-g82a5585 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)


Setup complete ✅ (8 CPUs, 51.0 GB RAM, 38.9/166.8 GB disk)


In [ ]:
%%writefile /content/yolov5/data/coco128.yaml
# Please insure that your custom_dataset are put in same parent dir with YOLOv6_DIR
train: /content/content/drive/MyDrive/Yolo_v6/datasets_usados_estructurados/custom_dataset/images/train/ # train images
val: /content/content/drive/MyDrive/Yolo_v6/datasets_usados_estructurados/custom_dataset/images/val/ # val images
test: /content/content/drive/MyDrive/Yolo_v6/datasets_usados_estructurados/custom_dataset/images/test/ # test images (optional)


# Classes
nc: 10  # number of classes


names:
  0: pedestrian
  1: people
  2: bicycle
  3: car
  4: van
  5: truck
  6: tricycle
  7: awning-tricycle
  8: bus
  9: motor

Overwriting /content/yolov5/data/coco128.yaml


In [ ]:
%%writefile /content/yolov5/models/yolov5s.yaml
# YOLOv5 🚀 by Ultralytics, GPL-3.0 license

# Parameters
nc: 10  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, C3, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, C3, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

Overwriting /content/yolov5/models/yolov5s.yaml


In [3]:
# Train YOLOv5s on COCO128 for 3 epochs
!python train.py --img 640 --batch 16 --epochs 100 --data coco128.yaml --weights '' --cfg yolov5s.yaml --cache

train: weights=, cfg=yolov5s.yaml, data=coco128.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.2-224-g82a5585 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.

In [ ]:
from utils.plots import plot_results
plot_results('/content/yolov5/runs/train/exp/results.csv')  # plot 'results.csv' as 'results.png'

In [ ]:
import shutil
shutil.copytree("/content/yolov5/runs/train","/content/drive/MyDrive/Yolo_v6/results")

'/content/drive/MyDrive/Yolo_v6/results_1'

In [5]:
!python detect.py --weights /content/drive/MyDrive/Yolo_v6/results_1/exp/weights/best.pt --img 640 --conf 0.5 --source /content/probe_1.jpg

detect: weights=['/content/drive/MyDrive/Yolo_v6/results_1/exp/weights/best.pt'], source=/content/probe_1.jpg, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v6.2-224-g82a5585 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 157 layers, 7037095 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 /content/probe_1.jpg: 384x640 1 people, 6 cars, 12.2ms
Speed: 0.4ms pre-process, 12.2ms inference, 1.3ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/exp2
